# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.95,96,99,7.23,TF,1712861770
1,1,iqaluit,63.7506,-68.5145,-2.15,68,100,2.06,CA,1712862191
2,2,adamstown,-25.0660,-130.1015,21.34,55,2,7.33,PN,1712861766
3,3,qamdo,31.1667,97.2333,-0.83,54,90,1.85,CN,1712862191
4,4,margaret river,-33.9500,115.0667,13.39,76,5,5.31,AU,1712861781


Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 600,
    color= "City",
    size= "Humidity",
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[
    (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Wind Speed'] >= 0) & (city_data_df['Wind Speed'] <= 5) &
    (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values

# Display sample data
ideal_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,naliya,23.2667,68.8333,25.30,81,0,4.45,IN,1712862197
88,88,faro,37.0194,-7.9322,22.88,53,0,2.57,PT,1712861691
138,138,siwa oasis,29.2041,25.5195,23.79,18,0,3.68,EG,1712862203
178,178,al qusiyah,27.4402,30.8184,22.89,29,0,4.37,EG,1712862207
181,181,crane,31.3974,-102.3501,24.27,17,0,4.51,US,1712861771
204,204,maile,-38.9466,-72.3727,21.34,28,0,2.28,CL,1712862210
218,218,pisco,-13.7000,-76.2167,25.03,73,0,4.12,PE,1712862205
245,245,al bawiti,28.3492,28.8659,21.63,29,0,4.00,EG,1712862213
326,326,california city,35.1258,-117.9859,26.27,26,0,3.09,US,1712862220
333,333,ivins,37.1686,-113.6794,25.07,14,0,0.89,US,1712862221


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ""



# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
61,naliya,IN,23.2667,68.8333,81,
88,faro,PT,37.0194,-7.9322,53,
138,siwa oasis,EG,29.2041,25.5195,18,
178,al qusiyah,EG,27.4402,30.8184,29,
181,crane,US,31.3974,-102.3501,17,
204,maile,CL,-38.9466,-72.3727,28,
218,pisco,PE,-13.7000,-76.2167,73,
245,al bawiti,EG,28.3492,28.8659,29,
326,california city,US,35.1258,-117.9859,26,
333,ivins,US,37.1686,-113.6794,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20}

#Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
naliya - nearest hotel: No hotel found
faro - nearest hotel: Best Western Hotel Dom Bernardo
siwa oasis - nearest hotel: فندق الكيلانى
al qusiyah - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
maile - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
al bawiti - nearest hotel: Old Oasis Hotel
california city - nearest hotel: Best Western California City Inn & Suites
ivins - nearest hotel: Crescent Moon Inn
saint-pierre - nearest hotel: Tropic Hotel
al quwayrah - nearest hotel: Malakot of Wadi Rum
cambria - nearest hotel: Olallieberry Inn
ormara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
61,naliya,IN,23.2667,68.8333,81,No hotel found
88,faro,PT,37.0194,-7.9322,53,Best Western Hotel Dom Bernardo
138,siwa oasis,EG,29.2041,25.5195,18,فندق الكيلانى
178,al qusiyah,EG,27.4402,30.8184,29,No hotel found
181,crane,US,31.3974,-102.3501,17,No hotel found
204,maile,CL,-38.9466,-72.3727,28,No hotel found
218,pisco,PE,-13.7000,-76.2167,73,La Portada
245,al bawiti,EG,28.3492,28.8659,29,Old Oasis Hotel
326,california city,US,35.1258,-117.9859,26,Best Western California City Inn & Suites
333,ivins,US,37.1686,-113.6794,14,Crescent Moon Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    scale = 2,
    frame_width = 800,
    frame_height = 600,
    color= "City",
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"],
    tiles = "OSM")

# Display the map
map_plot_2



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)